<a href="https://colab.research.google.com/github/pavelmurad/MURAD-HOSSEN-/blob/main/p_15_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"

In [32]:
import dolfinx

In [33]:

import ufl
import numpy as np

from petsc4py import PETSc
from mpi4py import MPI

from dolfinx import fem, mesh, io, plot
from petsc4py.PETSc import ScalarType
from dolfinx.fem.petsc import LinearProblem
from ufl import inner, grad, dx


In [34]:
# Set manually the values for meshsize and acoef
N = 100
acoef = 0.1

msh = mesh.create_unit_square(MPI.COMM_WORLD, N, N)
x = ufl.SpatialCoordinate(msh)
V = fem.functionspace(msh, ("Lagrange", 1))

In [35]:


# Create boundary condition
fdim = msh.topology.dim - 1
facet = mesh.locate_entities_boundary(msh, dim=fdim,
                                       marker=lambda x: np.isclose(x[0], 0.0))


In [36]:
dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facet)

In [37]:
g = fem.Function(V)
g.interpolate(lambda x : (x[1]**2)*(1.0-(2.0/3.0)*x[1]))


In [38]:
bc = fem.dirichletbc(g, dofs=dofs)

In [39]:
#Defining variational problem

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

f = fem.Function(V)
f.interpolate(lambda x: 1 - x[0])
bee = fem.Constant(msh,ScalarType((1,0)))
a = acoef*inner(grad(u),grad(v))*dx  + inner(bee,grad(u))*v*dx
L = inner(f,v)*dx

In [40]:
# compute solution
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [41]:
with io.XDMFFile(msh.comm, "advection.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)